# Part-1 : Fetch the reviews of the product provided by the user

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Where the Main and sub Review are present

In [4]:
def fetch_reviews_multiple(url, start=1, end=20):
    reviews_list = []

    for i in range(start, end + 1):
        url_page = f"{url}&page={i}"
        response = requests.get(url_page)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        reviews_container = soup.find_all("div", class_="cPHDOP col-12-12")
        if not reviews_container:
            continue  # Skip pages with no reviews

        review_divs = soup.find_all("div", class_="ZmyHeo")
        for div in review_divs:
            review_text = div.div.text.strip()
            review_text = review_text.replace("READ MORE","")
            reviews_list.append(review_text)
            
    return pd.DataFrame({'review': reviews_list})

# Where the Single Review is present

In [7]:
def fetch_reviews_single(url, start=1, end=20):
    reviews_list = []

    for i in range(start, end + 1):
        url_page = f"{url}&page={i}"
        response = requests.get(url_page)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        reviews_container = soup.find_all("div", class_="ZmyHeo MDcJkH")
        if not reviews_container:
            continue  # Skip pages with no reviews

        review_divs = soup.find_all("div", class_="_11pzQk")
        for div in review_divs:
            # review_text = div.div.text.strip()
            review_text = div.get_text(strip=True)
            review_text = review_text.replace("READ MORE","")
            reviews_list.append(review_text)
            
    return pd.DataFrame({'review': reviews_list})

# Final function

In [93]:
def Fetch_Review(url,Type='multiple'):
    df_final = pd.DataFrame()
    if Type == 'single':
            df1 = fetch_reviews_single(url)
            df_final = df1
    elif Type == 'multiple':
            df2 = fetch_reviews_multiple(url)
            df_final = df2
    else:
        print('wrong type')
    return df_final

# user input which show in frontend

In [117]:
url = input("Enter link :")
dff = Fetch_Review(url,Type='single')

Enter link : https://www.flipkart.com/imsa-moda-lingerie-set/product-reviews/itmc76d7e675de15?pid=LINGNY8ZJUBE58SN&lid=LSTLINGNY8ZJUBE58SNDBLQMD&marketplace=FLIPKART


In [119]:
dff

review
0                                        Nice pic 😍☺️
1   Absolutely made gorgeous, my jiju loves me in it.
2                                        Good product
3                                  Awesome product 🔥🔥
4   Not good.. size and colour is good. Bra qualit...
..                                                ...
75  Atcha hai but if you have heavy breast then it...
76                                       Nice product
77                                              Ok ok
78                                               Good
79                            Panty Size Is Too Short

[80 rows x 1 columns]

# Part-2 : Check the sentiment of the reviews fetched in part 1 

In [49]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# The Sentiment_model function is replaced my the actual traied LSTM model 

In [52]:
def sentiment_model(text):
    sia = SentimentIntensityAnalyzer()
    over_all_polarity = sia.polarity_scores(text)
    if over_all_polarity['compound'] >= 0.05:
        return "positive"
    elif over_all_polarity['compound'] <= -0.05:
        return "negative"
    else:
        return "neutral"

# This is the Function use in actual backend for check the polarity of the reviews

In [55]:
def Check_sentiment(df):
    df_with_polarity = df.copy()
    df_with_polarity['sentiment'] = df_with_polarity['review'].apply(sentiment_model)
    return df_with_polarity

In [121]:
final_df = Check_sentiment(dff)

In [123]:
final_df

review sentiment
0                                        Nice pic 😍☺️  positive
1   Absolutely made gorgeous, my jiju loves me in it.  positive
2                                        Good product  positive
3                                  Awesome product 🔥🔥  positive
4   Not good.. size and colour is good. Bra qualit...  positive
..                                                ...       ...
75  Atcha hai but if you have heavy breast then it...   neutral
76                                       Nice product  positive
77                                              Ok ok  positive
78                                               Good  positive
79                            Panty Size Is Too Short   neutral

[80 rows x 2 columns]

In [125]:
final_df['sentiment'].value_counts()

sentiment
positive    59
negative    15
neutral      6
Name: count, dtype: int64